In [42]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd


def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities


df = pd.read_csv('output/cities.csv')

In [5]:
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]

In [ ]:
from facebook_business.adobjects.customaudience import CustomAudience

dir(CustomAudience.Field)

In [ ]:
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

# include entry survey????

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        }
    }

    id_ = df.disthash.iloc[0]

    return { 'id': id_,
             'metadata': {'stratumid': id_},
             'id_field': 'stratumid',
             'facebook_targeting': targeting, 
             'quota': quota, 
             'creative_group': df.creative_group.iloc[0],  
             'custom_audience': df.include_audience.iloc[0],
             'shortcodes': ['baselineng', 'baselinehin', 'baselineodi'],
             'target_questions': [
                 {
                     'name': 'occupation',
                     'ref': '4fc929c7-132d-49b1-a164-515e5cc9064f',
                     'op': 'answered',
                     'field': 'response',
                     'value': None
                 },
             ]}


df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 100, df)).tolist()
    